<a href="https://colab.research.google.com/github/electricShoes/ML/blob/main/pytorch_4%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import platform
import matplotlib.pyplot as plt
import numpy as np


In [6]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, 5) #in, out, filtersize
    self.pool = nn.MaxPool2d(2, 2) #2x2 pooling
    self.conv2 = nn.Conv2d(32, 64, 5)
    self.fc1 = nn.Linear(64 * 4 * 4, 1000)
    self.fc2 = nn.Linear(1000, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 64 * 4 * 4)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x    

In [7]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
  model.train()
  running_loss =0.0
  criterion = nn.CrossEntropyLoss() #defalut is mean of mini-batchsamples
  
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), running_loss/log_interval))
      running_loss =0.0

In [11]:
def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  criterion = nn.CrossEntropyLoss(reduction='sum') #add all samples in a mini-batch
  
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      output = model(data)
      loss = criterion(output, target)
      test_loss += loss.item()
      pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

In [13]:
def main():
  epochs = 5
  learning_rate = 0.001
  batch_size = 32
  test_batch_size=1000
  log_interval =100
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  nThreads = 1 if use_cuda else 2
  if platform.system() == 'Windows':
    nThreads =0 #if you use windows

  transform = transforms.Compose(
      [transforms.ToTensor(),
      transforms.Normalize((0.5,), (0.5,))])

  # datasets
  trainset = torchvision.datasets.FashionMNIST('./data',
      download=True,
      train=True,
      transform=transform)

  testset = torchvision.datasets.FashionMNIST('./data',
      download=True,
      train=False,
      transform=transform)

  train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=nThreads)

  test_loader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size, shuffle=False, num_workers=nThreads)


  # constant for classes
  classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

  model = Net().to(device)

  #optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
  optimizer = optim.Adam(model.parameters(),lr=learning_rate)

  for epoch in range(1, epochs + 1):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


In [14]:
if __name__ == '__main__':
  main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.023014
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.895639
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.597007
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.532160
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.482058
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.430627
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.408088
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.395943
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.403735
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.379228
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.371141
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.353360
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.342107
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.358040
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.324756
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.321192
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.338156
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.315477
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.307608

Test set: Average loss: 0.3367, Accuracy: 8752/10000 (88%)